In [ ]:
import sys

In [ ]:
print (sys.version)

In [ ]:
!sudo chmod 777 -R /opt/Anaconda3-5.2.0-Linux-x86_64/lib/python3.6/
!sudo chmod 777 -R /opt/Anaconda3-5.2.0-Linux-x86_64/bin/

In [ ]:
!pip install -U pip

In [ ]:
!pip install pykafka

In [ ]:
# example my-cluster-kafka-external-bootstrap = "185.86.147.123:9094"  
# replace "185.86.147.123:9094" with your value
#you can find it with: kubectl get service my-cluster-kafka-external-bootstrap -n kafka

my_cluster_kafka_external_bootstrap_ip = "185.86.144.156:9094"
inventory_topic = b'dbserver1.inventory.customers'

In [ ]:
from pykafka import KafkaClient
client = KafkaClient(hosts=my_cluster_kafka_external_bootstrap_ip)

In [ ]:
client.topics

In [ ]:
topic = client.topics[inventory_topic]

In [ ]:
##stop this cell manually after testing
consumer = topic.get_simple_consumer(reset_offset_on_start=True)
for message in consumer:
    if message is not None:
        print (message.offset, message.value)

In [ ]:
# you can test sending messages
with topic.get_sync_producer() as producer:
    for i in range(4):
        producer.produce(b'test message')

In [ ]:
!pip install mysql-connector-python

In [ ]:
#replace with ip of your mysql instance
#you can find it with: kubectl get service mysql-externallb -n kafka
mysql_external_ip = '185.86.144.162'

In [ ]:
host_mysql = mysql_external_ip
database_mysql = 'inventory'
user_mysql = 'root'
password_mysql = 'debezium'

In [ ]:
#testing connection
import mysql.connector

connection = mysql.connector.connect(host=host_mysql,
                                         database=database_mysql,
                                         user=user_mysql,
                                         password=password_mysql)

mySql_select = """show databases;"""

cursor = connection.cursor()
db_Info = connection.get_server_info()
print("Connected to MySQL Server version ", db_Info)
cursor.execute("select database();")
record = cursor.fetchone()
print("You're connected to database: ", record)

connection.close()


In [ ]:
import mysql.connector

connection = mysql.connector.connect(host=host_mysql,
                                         database=database_mysql,
                                         user=user_mysql,
                                         password=password_mysql)

cursor = connection.cursor()
cursor.execute("show tables;")
record = cursor.fetchall()
print(record)

connection.close()

In [ ]:
#generate random records to MySQL

import random
import time
import mysql.connector

connection = mysql.connector.connect(host=host_mysql,
                                         database=database_mysql,
                                         user=user_mysql,
                                         password=password_mysql)

sleep_time = 1
number_of_records = 5

regions = ['Moscow', 'Piter', 'Kazan', 'Saratov', 'Voronezh']
products = ['Bread', 'Milk', 'Juice', 'Umbrella', 'Echpochmak']
cursor = connection.cursor()
db_Info = connection.get_server_info()
print("Connected to MySQL Server version ", db_Info)


def random_insert():

    query = f"""INSERT INTO inventory.sales(client_id, region, product_name, quantity, price)
        VALUES({random.randint(1, 50)},
        '{random.choice(regions)}', 
        '{random.choice(products)}', 
        {random.randint(1, 10)}, 
        {random.randint(1, 1000)});"""
    cursor.execute(query)
    connection.commit()
    time.sleep(sleep_time)
    print (query)

for i in range(number_of_records):
    random_insert()
    

In [ ]:
#postgres part

In [ ]:
!pip install psycopg2-binary

In [ ]:
#replace host_pg value with ip of your base
#create user for NiFi in postgre from MCS UI https://mcs.mail.ru/app/services/databases/list/
host_pg = '185.86.146.66'
database_pg = 'PostgreSQL-4147'
user_pg = 'nifi'
password_pg = 'dVY4ddqXdnQ8EHj1'

In [ ]:
#test connection
import psycopg2

conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()

print('PostgreSQL database version:')
cursor.execute('SELECT version()')
# display the PostgreSQL database server version
db_version = cursor.fetchone()
print(db_version)
# close the communication with the PostgreSQL
cursor.close()
conn.close()


In [ ]:
#create target table in Postgre
conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()
cursor.execute("""CREATE TABLE sales (client_id INT,
region VARCHAR (100),
product_name VARCHAR(100),
quantity INT,
price  INT,
created_at TIMESTAMP
);
""")
cursor.close()
conn.commit()
conn.close()


In [ ]:
#check if table exist
conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()
cursor.execute("""SELECT
    table_schema || '.' || table_name
FROM
    information_schema.tables
WHERE
    table_type = 'BASE TABLE'
AND
    table_schema NOT IN ('pg_catalog', 'information_schema');""")

for row in cursor:
    print(row)

cursor.close()
conn.close()

In [ ]:
#test insert
conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()
cursor.execute("""INSERT INTO public.sales(client_id, region, product_name, quantity, price, created_at)
VALUES
(1,'Moscow', 'Milk', 3, 85, now()),
(2,'Piter', 'Umbrella', 3, 385, now()),
(3,'Kazan', 'Ichpochmak', 3, 123, now()),
(4,'Saratov', 'Bread', 3, 8, now()),
(5,'Voronezh', 'Juice', 3, 42, now())
;""")
cursor.close()
conn.commit()
conn.close()

In [ ]:
#test result of insert
conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()
cursor.execute("""SELECT * FROM public.sales;""")

for row in cursor:
    print(row)

cursor.close()
conn.close()

In [ ]:
#truncate if you wish
conn = psycopg2.connect(dbname=database_pg, user=user_pg, 
                        password=password_pg, host=host_pg)
cursor = conn.cursor()
cursor.execute("""TRUNCATE public.sales;""")
cursor.close()
conn.commit()
conn.close()

In [ ]:
#Next step. We will generate insert records to MySQL
#Than read records from Kafka with Nifi and replicate to Postgre with Nifi


In [ ]:
#RUN THIS after you created and configured Nifi
#generate random records to MySQL

import random
import time
import mysql.connector

connection = mysql.connector.connect(host=host_mysql,
                                         database=database_mysql,
                                         user=user_mysql,
                                         password=password_mysql)

sleep_time = 1
number_of_records = 5000

regions = ['Moscow', 'Piter', 'Kazan', 'Saratov', 'Voronezh']
products = ['Bread', 'Milk', 'Juice', 'Umbrella', 'Echpochmak']
cursor = connection.cursor()
db_Info = connection.get_server_info()
print("Connected to MySQL Server version ", db_Info)


def random_insert():

    query = f"""INSERT INTO inventory.sales(client_id, region, product_name, quantity, price)
        VALUES({random.randint(1, 50)},
        '{random.choice(regions)}', 
        '{random.choice(products)}', 
        {random.randint(1, 10)}, 
        {random.randint(1, 1000)});"""
    cursor.execute(query)
    connection.commit()
    time.sleep(sleep_time)
    print (query)

for i in range(number_of_records):
    random_insert()
    